## Assignment 4
#### Instruction
Utilizing the methodology outlined in this chapter, demonstrate the process of ranking spelling correction candidates for a single mistyped word. Your task involves the following key components:
- Choose a word that is misspelled, ensuring that all candidate corrections are only one edit distance away from this selected word.
- Generate at least four viable candidates for spelling correction. These candidates should be closely related to your chosen mistyped word, differing by only a single edit in spelling.
- Initially, apply the IULA to automate the ranking process as much as possible. Analyze how each candidate ranks in terms of likelihood of being the correct spelling correction.
- Extract a different corpus -- Apply the same counting technique you used with Norvig's approach to a new corpus rather than IULA.

In [100]:
import pandas as pd
import numpy as np
import re
import os
import string
from ordered_set import OrderedSet
from sklearn.feature_extraction.text import CountVectorizer

import nltk
nltk.download("stopwords")
nltk.download("punkt")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/tkthanatorn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/tkthanatorn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### IULA Corpus

In [101]:
topdir = "../../data/iula"
all_content = []
for dirpath, dirname, filename in os.walk(topdir):
    for name in filename:
        if name.endswith("plain.txt"):
            with open(os.path.join(dirpath, name)) as f:
                all_content.append(f.read())

In [102]:
def preProcess(texts: list[str], stop_dict: dict) -> list[str]:
    data = [
        s.translate(str.maketrans("", "", string.punctuation + "\xa0")) for s in texts
    ]
    data = [s.lower() for s in data]
    data = [
        s.translate(str.maketrans(string.whitespace, " " * len(string.whitespace), ""))
        for s in data
    ]

    tokenized = [word_tokenize(s) for s in data]
    concatenated = np.unique(np.concatenate(tokenized))
    stem_cache = {}
    ps = PorterStemmer()
    for s in concatenated:
        stem_cache[s] = ps.stem(s)
    
    def custom_processor(s: str):
        ps = PorterStemmer()
        s = re.sub(r"[^A-Za-z]", " ", s)
        s = re.sub(r"\s+", " ", s)
        s = word_tokenize(s)
        s = list(OrderedSet(s) - stop_dict)
        s = [word for word in s if len(word) > 2]
        s = [stem_cache[w] if w in stem_cache else ps.stem(w) for w in s]
        s = " ".join(s)
        return s

    data = [custom_processor(s) for s in data]
    return data

stop_dict = set(stopwords.words("english"))
preprocessed = preProcess(all_content, stop_dict)

##### Generate candidate function

In [103]:
def generate_candidates(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    swaps = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + swaps + replaces + inserts)

generate candidate words that are one edit distance

In [104]:
word = "development"
candidates = list(generate_candidates(word))[:4]
candidates[0] = word
candidates

['development', 'developmfnt', 'dlevelopment', 'dzvelopment']

fit and transform IULA corpus with CountVectorizer model

In [105]:
vectorizer = CountVectorizer()
frequency = vectorizer.fit_transform(preprocessed)
frequency = pd.DataFrame(frequency.todense(), columns=vectorizer.get_feature_names_out()).sum()
frequency.head()

aaa        1
aaaaaa     1
aalborg    2
aarhu      1
aaron      3
dtype: int64

transform the candidates words with processed model

In [106]:
transformed_candidates = [
    vectorizer.inverse_transform(vectorizer.transform([candidate]))
    for candidate in candidates
]

transformed_frequency = pd.Series(
    [
        frequency.T.loc[tq[0]].values[0] if len(tq[0]) > 0 else 0
        for tq in transformed_candidates
    ],
    index=candidates,
)

transformed_frequency.head()

development     22
developmfnt      0
dlevelopment     0
dzvelopment      0
dtype: int64

ranking the candidate words

In [107]:
IULA = pd.DataFrame(transformed_frequency, columns=["frequency"])
IULA_TOTAL_FREQUENCY = IULA["frequency"].sum()
IULA["P(w)"] = IULA["frequency"] / IULA_TOTAL_FREQUENCY
IULA["rank"] = IULA["frequency"].rank(ascending=False).astype(int)

IULA.head()

,frequency,P(w),rank
development,22,1.0,1
developmfnt,0,0.0,3
dlevelopment,0,0.0,3
dzvelopment,0,0.0,3


#### Norvig Corpus

In [108]:
norvig_path = "../../data/norvig.txt"
norvig = []
with open(norvig_path) as f:
    norvig = sent_tokenize(f.read())

norvig = preProcess(norvig, stop_dict)

In [121]:
word = "prize"
candidates = list(generate_candidates(word))[:4]
candidates[0] = word
candidates

['prize', 'wprize', 'prbze', 'priuze']

In [122]:
norvig_vectorizer = CountVectorizer()
norvig_frequency = norvig_vectorizer.fit_transform(norvig)
norvig_frequency = pd.DataFrame(norvig_frequency.todense(), columns=norvig_vectorizer.get_feature_names_out())
norvig_frequency = norvig_frequency.sum()
norvig_frequency.head()

aaron       1
abandon     6
abat        1
abbot       1
aberdeen    2
dtype: int64

In [123]:
norvig_transformed_candidates = [
    norvig_vectorizer.inverse_transform(norvig_vectorizer.transform([candidate]))
    for candidate in candidates
]

norvig_transformed_frequency = pd.Series(
    [
        norvig_frequency.T.loc[tq[0]].values[0] if len(tq[0]) > 0 else 0
        for tq in norvig_transformed_candidates
    ],
    index=candidates,
)

norvig_transformed_frequency.head()

prize     5
wprize    0
prbze     0
priuze    0
dtype: int64

In [124]:
NORVIG = pd.DataFrame(norvig_transformed_frequency, columns=["frequency"])
NORVIG_TOTAL_FREQUENCY = NORVIG["frequency"].sum()
NORVIG["P(w)"] = NORVIG["frequency"] / NORVIG_TOTAL_FREQUENCY
NORVIG["rank"] = NORVIG["frequency"].rank(ascending=False).astype(int)

NORVIG.head()

,frequency,P(w),rank
prize,5,1.0,1
wprize,0,0.0,3
prbze,0,0.0,3
priuze,0,0.0,3
